# MLP Gradient Descent - Default_Dataset

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 실습용 데이터 설정

* pandas DataFrame
 * Default.csv

In [ ]:
import pandas as pd

DF = pd.read_csv('https://raw.githubusercontent.com/rusita-ai/pyData/master/Default.csv')

DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [ ]:
DF.head()

,default,student,balance,income
0,No,No,729.526495,44361.62507
1,No,Yes,817.180407,12106.13470
2,No,No,1073.549164,31767.13895
3,No,No,529.250605,35704.49394
4,No,No,785.655883,38463.49588


# I. Data Preprocessing

> ## 1) DataFrame to Array

In [ ]:
X = DF[['balance', 'income']]
y = DF['default']

* DataFrame to Array

In [ ]:
X_input = X.values
y_output = y.values

> ## 2) Integer Encoding

* LabelEncoder( )
 - ['NO', 'Yes'] to [0, 1]

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder =  LabelEncoder()
y_output = encoder.fit_transform(y_output)

y_output[400:420]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

> ## 2) Train & Test Split

* 7: 3

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_input, y_output,
                                                    test_size = 0.3,
                                                    random_state = 2045)

print('Train Data : ', X_train.shape, y_train.shape)
print('Test Data : ', X_test.shape, y_test.shape)

Train Data :  (7000, 2) (7000,)
Test Data :  (3000, 2) (3000,)


# II. 수치미분(Numerical Derivative)

> ## 1) Import numpy

In [ ]:
import numpy as np 

> ## 2) gradient( ) 함수 정의

* 다변수 함수의 수치미분

In [ ]:
def gradient(machine, param):

    if param.ndim == 1:
        temp_param = param
        delta = 0.00005
        learned_param = np.zeros(param.shape)
        
        for index in range(len(param)):
            target_param = float(temp_param[index])
            temp_param[index] = target_param + delta            
            param_plus_delta = machine(temp_param)
            temp_param[index] = target_param - delta  
            param_minus_delta = machine(temp_param)
            learned_param[index] = (param_plus_delta - param_minus_delta ) / (2 * delta)
            temp_param[index] = target_param

        return learned_param
        

    elif param.ndim == 2:
        temp_param = param
        delta = 0.00005
        learned_param = np.zeros(param.shape)
    
        rows = param.shape[0]
        columns = param.shape[1]
    
        for row in range(rows):
            for column in range(columns):
                target_param = float(temp_param[row, column])
                temp_param[row, column] = target_param + delta            
                param_plus_delta = machine(temp_param)
                temp_param[row, column] = target_param - delta            
                param_minus_delta = machine(temp_param)
                learned_param[row, column] = (param_plus_delta - param_minus_delta) / (2 * delta)
                temp_param[row, column] = target_param

        return learned_param

> ## 3) Activation - sigmoid( )

In [ ]:
def sigmoid(x):
    y_hat = 1 / (1 + np.exp(-x))
    return y_hat

# III. Model Training(Learning)

> ## 1) MLP_Classifier Class

In [ ]:
class MLP_Classifier:
    
    def __init__(self, problem_Type, X_input, y_output):  

# problem_Type 문자열 지정 Member      
        self.Type = problem_Type
        
# X_input, y_output Member 초기화
        self.X_input = X_input.reshape(-1, 2)
        self.y_output = y_output.reshape(-1, 1)
        
# W_1, b_1 Member 초기화
        self.W_1 = np.random.rand(2, 2)  
        self.b_1 = np.random.rand(2)

# W_2, b_2 Member 초기화
        self.W_2 = np.random.rand(2, 1)  
        self.b_2 = np.random.rand(1)

# learning_rate Member 지정
        self.learning_rate = 0.01

# Cost_Function(CEE) Method
    def cost_func(self):

        z_1 = np.dot(self.X_input, self.W_1) + self.b_1     # Hidden Layer
        a_1 = sigmoid(z_1)                                
        
        z_2 = np.dot(a_1, self.W_2) + self.b_2            # Output Layer
        y_hat = sigmoid(z_2)                        

        delta = 0.00001
        return -np.sum(self.y_output * np.log(y_hat + delta) + (1 - self.y_output) * np.log((1 - y_hat) + delta))      

# Learning Method
    def fit(self):
        machine = lambda x : self.cost_func()
        print('Initial Cost = ', self.cost_func())
        
        for step in  range(10001):
            self.W_1 = self.W_1- self.learning_rate * gradient(machine, self.W_1)
            self.b_1 = self.b_1 - self.learning_rate * gradient(machine, self.b_1)

            self.W_2 = self.W_2 - self.learning_rate * gradient(machine, self.W_2)
            self.b_2 = self.b_2 - self.learning_rate * gradient(machine, self.b_2)
    
            if (step % 1000 == 0):
                print('Step = ', step, 'Cost = ', self.cost_func())
                
# Predict Method
    def predict(self, input_data):
        
        z_1 = np.dot(input_data, self.W_1) + self.b_1     # Hidden Layer
        a_1 = sigmoid(z_1)                                
        
        z_2 = np.dot(a_1, self.W_2) + self.b_2            # Output Layer
        y_prob = sigmoid(z_2)                             


        if y_prob > 0.5:
            result = 1
        else:
            result = 0
    
        return y_prob, result

> ## 2) Default.fit( )

* 약 5분

In [ ]:
%%time

Default = MLP_Classifier('Binary_Classification', X_train, y_train)

Default.fit()

Initial Cost =  7306.1991762444595
Step =  0 Cost =  2636.39222181673
Step =  1000 Cost =  2636.39222181673
Step =  2000 Cost =  2636.39222181673
Step =  3000 Cost =  2636.39222181673
Step =  4000 Cost =  2636.39222181673
Step =  5000 Cost =  2636.39222181673
Step =  6000 Cost =  2636.39222181673
Step =  7000 Cost =  2636.39222181673
Step =  8000 Cost =  2636.39222181673
Step =  9000 Cost =  2636.39222181673
Step =  10000 Cost =  2636.39222181673
CPU times: user 5min, sys: 4min 2s, total: 9min 2s
Wall time: 4min 53s


> ## 3) Default.predict( )

In [ ]:
y_hat = []

for input_data in X_test:
    (sigmoid_val, logical_val) = Default.predict(input_data) 
    print(input_data, ' = ', logical_val)

    y_hat.append(logical_val)

[ 1051.247162 23200.97099 ]  =  0
[  887.594632 59416.15976 ]  =  0
[ 1134.344472 41748.18345 ]  =  0
[  272.9878407 63566.36599  ]  =  0
[    0.      32481.54005]  =  0
[ 1291.915263 17910.06461 ]  =  0
[  904.0563158 23188.71514  ]  =  0
[ 1077.020824 56862.06694 ]  =  0
[  499.1739193 44773.57533  ]  =  0
[  744.9142665 15711.71642  ]  =  0
[ 1579.359465 50216.57163 ]  =  0
[ 1282.567571 36558.83792 ]  =  0
[  413.5990058 31466.22059  ]  =  0
[ 1347.635595 58953.09267 ]  =  0
[  794.9731014 21590.96187  ]  =  0
[  439.8374803 24452.6466   ]  =  0
[  908.1844428 37319.62257  ]  =  0
[  563.2901332 22323.83355  ]  =  0
[  258.6296725 33327.93778  ]  =  0
[  860.0343646 33740.69911  ]  =  0
[  811.7699461 46076.30224  ]  =  0
[ 1030.210058 40711.45317 ]  =  0
[  843.0178966 12268.93352  ]  =  0
[  583.5997492 15179.68613  ]  =  0
[  912.3904235 26857.1719   ]  =  0
[  431.159894 51727.54032 ]  =  0
[  425.8446454 15324.758    ]  =  0
[  571.8863591 15632.76928  ]  =  0
[  993.0495382 2

> ## 4) Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_hat)

array([[2896,    0],
       [ 104,    0]])

> ## 5) Classification Report

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_hat, 
                            target_names = ['0', '1'],
                            digits = 5))

              precision    recall  f1-score   support

           0    0.96533   1.00000   0.98236      2896
           1    0.00000   0.00000   0.00000       104

    accuracy                        0.96533      3000
   macro avg    0.48267   0.50000   0.49118      3000
weighted avg    0.93187   0.96533   0.94831      3000



# 
# 
# 
# The End
# 
# 
# 